In [1]:
# MSDS 422, Section 58, Assignment 4, Alan Kessler
# Python 3.5 on Mac OS 10.13.5 edited in Atom
# Demonstrates use of random forests

import pandas as pd
import numpy as np
import warnings
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages

# Suppress scipy runtime warning
warnings.filterwarnings(action='ignore', module='scipy',
                        message='^internal gelsd')

# Set random seed to reproduce results
seed = 2107
# Set number of cross-validation splits
splits = 5

# Load the data
train = pd.read_csv('boston.csv')

# Drop the neighborhood variable based on assignment instructions
train.drop(['neighborhood'], axis=1, inplace=True)

# Define scaled predictor variables as an array for sklearn use
X = StandardScaler().fit_transform(train.drop(['mv'], axis=1).values)

# Define log-target as an array for sklearn use
y = np.log(train['mv']).values

# Use KFold cross-validation to split data for training
cv = KFold(n_splits=splits, shuffle=False, random_state=seed)

# Construct a list of models to iterate over
models = [LinearRegression(), Ridge(), Lasso(), ElasticNet(),
          RandomForestRegressor(n_estimators=200, random_state=seed),
          RandomForestRegressor(max_features=1, n_estimators=200,
                                random_state=seed),
          RandomForestRegressor(max_features="log2", n_estimators=200,
                                random_state=seed),
          RandomForestRegressor(n_estimators=200, max_depth=2,
                                random_state=seed)]

# Initialize list to store stats from each fold and model
stats = []

# Iterate over the folds and models to save OoF RMSE for each
i = 0
for training, test in cv.split(X, y):
    fold = [i]
    for alg in models:
        scores = alg.fit(X[training], y[training]).predict(X[test])
        rmse = mean_squared_error(y[test], scores)**2
        fold.append(rmse)
    stats.append(fold)
    i += 1

# Create a dataframe of the OoF RMSE for each model
labels = ['RMSE Fold', 'Linear Regression', 'Ridge Regression',
          'Lasso Regression', 'Elastic Net Regression',
          'Random Forest - No Max', 'Random Forest - Max 1',
          'Random Forest - log2 Max', 'Random Forest - No Max 2 Depth']
results = pd.DataFrame(stats, columns=labels)

# Add mean and standard deviation to summary
m = results.mean().tolist()
m[0] = "Mean"
s = results.std().tolist()
s[0] = "Std Dev"

results_summary = results.append([pd.Series(m, index=labels),
                                  pd.Series(s, index=labels)],
                                 ignore_index=True).set_index('RMSE Fold')

# Save CV results to a CSV
results_summary.to_csv("results_summary.csv")

# Score the final model
final_model = RandomForestRegressor(max_features="log2", n_estimators=200,
                                    random_state=seed)
rf = final_model.fit(X, y)
train['RandomForest Prediction'] = np.exp(rf.predict(X))

# Save prediction results to a CSV
train.to_csv("results_scores.csv")

# Capture variable importance in final model
features = train.columns[:-2].tolist()
importance = rf.feature_importances_.tolist()
var_imp = pd.DataFrame.from_items([('Feature', features),
                                   ('Importance', importance)])
var_imp.sort_values(by=['Importance'], inplace=True, ascending=False)
var_imp.reset_index(inplace=True)
var_imp.drop(['index'], axis=1, inplace=True)

# Plot variable importance
fig, ax = plt.subplots(figsize=(10, 7))
ax = sns.barplot(x='Feature', y='Importance',
                 data=var_imp, errwidth=0,
                 color=sns.xkcd_rgb["denim blue"])
for index, row in var_imp.iterrows():
    ax.text(row.name, row['Importance']+0.001, round(row['Importance'], 2),
            color='black', ha="center")

plt.title("Random Forest Variable Importance")
eda = PdfPages('importance_visualization.pdf')
eda.savefig()
plt.close()
eda.close()


In [2]:
results_summary

,Linear Regression,Ridge Regression,Lasso Regression,Elastic Net Regression,Random Forest - No Max,Random Forest - Max 1,Random Forest - log2 Max,Random Forest - No Max 2 Depth
RMSE Fold,,,,,,,,
0,0.000346,0.000339,0.004336,0.004336,0.000408,0.000752,0.000418,0.000290
1,0.001485,0.001440,0.018730,0.018730,0.000959,0.001199,0.000438,0.002452
2,0.001004,0.000992,0.055127,0.055127,0.000529,0.002212,0.000679,0.002259
3,0.009121,0.009175,0.061669,0.061669,0.003728,0.006588,0.004118,0.007446
4,0.003213,0.003210,0.075685,0.075685,0.003638,0.005118,0.003586,0.005534
Mean,0.003034,0.003031,0.043109,0.043109,0.001852,0.003174,0.001848,0.003596
Std Dev,0.003565,0.003596,0.030198,0.030198,0.001684,0.002556,0.001842,0.002855


In [3]:
train

,crim,zn,indus,chas,nox,rooms,age,dis,rad,tax,ptratio,lstat,mv,RandomForest Prediction
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,4.98,24.0,24.527970
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,9.14,21.6,22.008152
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,4.03,34.7,34.210016
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,2.94,33.4,33.962552
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,5.33,36.2,34.592580
5,0.02985,0.0,2.18,0,0.458,6.430,58.7,6.0622,3,222,18.7,5.21,28.7,27.824078
6,0.08829,12.5,7.87,0,0.524,6.012,66.6,5.5605,5,311,15.2,12.43,22.9,21.942066
7,0.14455,12.5,7.87,0,0.524,6.172,96.1,5.9505,5,311,15.2,19.15,22.1,20.386613
8,0.21124,12.5,7.87,0,0.524,5.631,100.0,6.0821,5,311,15.2,29.93,16.5,16.704421
9,0.17004,12.5,7.87,0,0.524,6.004,85.9,6.5921,5,311,15.2,17.10,18.9,18.831364
